In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.stats import gaussian_kde

In [2]:
# Estilo visual
sns.set(style="whitegrid")
np.random.seed(42)

In [3]:
# Dados
data = np.random.normal(loc=0, scale=1, size=1000)
bins = np.linspace(-4, 4, 30)
bin_centers = 0.5 * (bins[1:] + bins[:-1])
heights, _ = np.histogram(data, bins=bins, density=True)

In [4]:
# KDE
kde = gaussian_kde(data)
x_vals = np.linspace(-4, 4, 300)
kde_vals = kde(x_vals)

In [5]:
# Configuração da figura
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(-4, 4)
ax.set_ylim(0, max(heights.max(), kde_vals.max()) + 0.05)
ax.set_title("Data Visualization: Histogram + KDE", fontsize=16, weight='bold')
ax.set_xlabel("Value")
ax.set_ylabel("Density")

plt.close()

In [6]:
# Elementos do gráfico
bars = ax.bar(bin_centers, np.zeros_like(bin_centers), width=0.25, alpha=0.6, color="#69b3a2")
kde_line, = ax.plot([], [], color='crimson', lw=3)
text = ax.text(-3.8, ax.get_ylim()[1] - 0.05, "", fontsize=13, weight='bold')

In [7]:
# Configurações de animação
frames_per_bar = 5
total_bar_frames = len(bin_centers) * frames_per_bar
total_kde_frames = 40
total_frames = total_bar_frames + total_kde_frames

In [8]:
def animate(frame):
    if frame < total_bar_frames:
        # Crescimento suave das barras
        bar_idx = frame // frames_per_bar
        progress = (frame % frames_per_bar) / frames_per_bar
        for i, bar in enumerate(bars):
            if i < bar_idx:
                bar.set_height(heights[i])
            elif i == bar_idx:
                bar.set_height(heights[i] * progress)
            else:
                bar.set_height(0)
        text.set_text("Construindo histograma suavemente...")
    else:
        # KDE da esquerda para a direita
        kde_frame = frame - total_bar_frames
        num_points = len(x_vals)
        reveal_idx = int((kde_frame + 1) / total_kde_frames * num_points)
        kde_line.set_data(x_vals[:reveal_idx], kde_vals[:reveal_idx])
        text.set_text("Curva KDE aparecendo da esquerda para a direita...")

    return bars, kde_line, text

In [9]:
# Criar animação
ani = FuncAnimation(fig, animate, frames=total_frames, interval=80, blit=False)

In [10]:
# Salvar como GIF (ou vídeo com ffmpeg, se quiser)
ani.save("datavisualization_hist_kde.gif", writer=PillowWriter(fps=12))